# Saltelli's example (1.3) & (1.27), Table 1.5.

In [23]:
import pandas
from scipy import stats

import sensitivity_analysis

In [24]:
def model(Z_0, Z_1, Z_2, Z_3, Ω_0, Ω_1, Ω_2, Ω_3):
    return (Ω_0 * Z_0 + Ω_1 * Z_1 + Ω_2 * Z_2 + Ω_3 * Z_3)


Z = {f'Z_{i}': stats.norm(0, i + 1)
     for i in range(4)}
Ω = {f'Ω_{i}': stats.norm((i + 1) * 0.5, i + 1) 
         for i in range(4)}
parameters = dict(**Z, **Ω)

In [25]:
def bold_sum(s):
    '''Made the 'Sum' row bold.'''
    is_sum = (s.index == 'Sum')
    return ['font-weight: bold' if v else '' for v in is_sum]


def display_Sobol_indexes(S, S_PE, S_T, S_T_PE, alpha=0.5):
    '''Display the Sobol' indexes S and S_T in a nice table.'''
    table = pandas.concat((S, S_PE, S_T, S_T_PE), axis='columns')
    CI_percent = 100 * (1 - alpha)
    columns = pandas.MultiIndex.from_product(
        (('First-order index ($S$)', 'Total-order index ($S_T$)'),
         ('Estimate', f'{CI_percent:g}% probable error')))
    table.set_axis(columns, axis='columns', inplace=True)
    table.loc['Sum'] = table.sum()
    # Don't show the sum of the probable errors.
    estimates = (columns.get_level_values(1) == 'Estimate')
    table.loc['Sum', ~estimates] = ''
    display(table.style.apply(bold_sum))


n_samples = 40000
(S, S_PE, S_T, S_T_PE) = sensitivity_analysis.sobol_index.get_indexes(
    model, parameters, n_samples)
display_Sobol_indexes(S, S_PE, S_T, S_T_PE)